**Questions** - Does IBM Speech-to-Text support Multi-speaker Diarization? 

# Multi-speaker Diarization

Please see the documentation [here](https://cloud.ibm.com/docs/speech-to-text?topic=speech-to-text-speaker-labels)

In [9]:
!pip install ibm_watson

In [10]:
import json
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [11]:
from IPython.display import Audio
filename="/project_data/data_asset/multiDiarization.wav"
Audio(filename)

In [12]:
import operator
credentials = {
        "apikey": "AjNp78CaBEXMFw9Ih-G1b6k3gGBQZ4lpallRLkKBU39D",
    "url": "https://api.au-syd.speech-to-text.watson.cloud.ibm.com/instances/ce0cb1f4-13a4-4c36-9ab6-d6905ea327a2"
  }


authenticator = IAMAuthenticator(credentials['apikey'])
speech_to_text = SpeechToTextV1(
    authenticator=authenticator
)

speech_to_text.set_service_url(credentials['url'])
speech_models = speech_to_text.list_models().get_result()
# print(json.dumps(speech_models, indent=2))

    
with open(filename, 'rb') as audio_file:
    speech_recognition_results = speech_to_text.recognize(
        audio=audio_file,
        content_type='audio/wav',
        speaker_labels=True,
        smart_formatting=True
    ).get_result()


transcripts = {0: "", 1: ""}

# Sort speaker labels according to timestamp
speaker_labels = sorted(speech_recognition_results['speaker_labels'], key=operator.itemgetter('from'))

# For each chunk of speech
for chunk in speech_recognition_results['results']:
    if 'alternatives' in chunk.keys():
        alternatives = chunk['alternatives'][0]
        if 'transcript' in alternatives:
            # Get the start time of the chunk
            start_time = alternatives['timestamps'][0][1]
            # Find the speaker for this chunk
            for speaker_label in speaker_labels:
                if speaker_label['from'] <= start_time <= speaker_label['to']:
                    current_speaker = speaker_label['speaker']
                    break
            # Add the transcript to the corresponding speaker
            transcripts[current_speaker] += alternatives['transcript'] + "\n"

for speaker, transcript in transcripts.items():
    print(f"Speaker {speaker} said: {transcript}")

Speaker 0 said: we have a long way to go 

Speaker 1 said: but we will have to wait 

